# 04 - Huấn luyện & đánh giá mô hình (Response)

So sánh mô hình, tuning siêu tham số, cross-validation, kết quả cuối cùng và gợi ý can thiệp.

## 1. Import thư viện & nạp dữ liệu
- Import các thư viện cần thiết
- Đọc dữ liệu từ file CSV
- Tách biến số và biến phân loại, chia train/test

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score, roc_auc_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
%matplotlib inline

data_path = '../healthcare-dataset-stroke-data.csv'
df = pd.read_csv(data_path)
target = 'stroke'
X = df.drop(columns=[target])
y = df[target]
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = [c for c in X.columns if c not in numeric_cols]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f'Train: {X_train.shape}, Test: {X_test.shape}')

Train: (4088, 11), Test: (1022, 11)


## 2. Pipeline tiền xử lý
- Xử lý missing values, chuẩn hóa biến số, mã hóa biến phân loại

In [2]:
preprocess = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_cols),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ],
    remainder='drop'
)

## 3. Định nghĩa pipeline cho các mô hình
- Logistic Regression, Decision Tree, Random Forest

In [3]:
pipe_lr = Pipeline([('preprocess', preprocess), ('model', LogisticRegression(max_iter=1000, class_weight='balanced'))])
pipe_dt = Pipeline([('preprocess', preprocess), ('model', DecisionTreeClassifier(class_weight='balanced', random_state=42))])
pipe_rf = Pipeline([('preprocess', preprocess), ('model', RandomForestClassifier(n_estimators=300, class_weight='balanced_subsample', random_state=42, n_jobs=-1))])

## 4. Cross-validation cho các mô hình
- Đánh giá accuracy và AUC với cross_val_score

In [4]:
def evaluate_with_cv(name, pipeline, cv=5):
    scores_acc = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='accuracy')
    scores_auc = cross_val_score(pipeline, X_train, y_train, cv=cv, scoring='roc_auc')
    print(f'{name}:')
    print(f'  Accuracy: {scores_acc.mean():.3f} (+/- {scores_acc.std():.3f})')
    print(f'  AUC: {scores_auc.mean():.3f} (+/- {scores_auc.std():.3f})')

evaluate_with_cv('Logistic Regression', pipe_lr, cv=3)
evaluate_with_cv('Decision Tree', pipe_dt, cv=3)
evaluate_with_cv('Random Forest', pipe_rf, cv=3)

Logistic Regression:
  Accuracy: 0.743 (+/- 0.021)
  AUC: 0.837 (+/- 0.019)
Decision Tree:
  Accuracy: 0.915 (+/- 0.001)
  AUC: 0.538 (+/- 0.032)
Random Forest:
  Accuracy: 0.951 (+/- 0.000)
  AUC: 0.811 (+/- 0.014)
Random Forest:
  Accuracy: 0.951 (+/- 0.000)
  AUC: 0.811 (+/- 0.014)


## 5. Đánh giá trên test set
- Hàm đánh giá tổng hợp: accuracy, AUC, precision, recall, F1

In [5]:
def final_eval(name, pipeline):
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    y_proba = pipeline.predict_proba(X_test)[:, 1] if hasattr(pipeline, 'predict_proba') else None
    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    print(f'\n{name} (Test set):')
    print(f'  Accuracy: {acc:.3f}, AUC: {auc:.3f}, Precision: {prec:.3f}')
    print(f'  Recall: {rec:.3f}, F1: {f1:.3f}')

print('\n=== Kết quả trên Test Set ===')
final_eval('Logistic Regression', pipe_lr)
final_eval('Decision Tree', pipe_dt)
final_eval('Random Forest', pipe_rf)


=== Kết quả trên Test Set ===

Logistic Regression (Test set):
  Accuracy: 0.750, AUC: 0.841, Precision: 0.140
  Recall: 0.800, F1: 0.238

Decision Tree (Test set):
  Accuracy: 0.935, AUC: 0.549, Precision: 0.214
  Recall: 0.120, F1: 0.154

Random Forest (Test set):
  Accuracy: 0.950, AUC: 0.789, Precision: 0.000
  Recall: 0.000, F1: 0.000

Random Forest (Test set):
  Accuracy: 0.950, AUC: 0.789, Precision: 0.000
  Recall: 0.000, F1: 0.000


## 6. Grid Search (tuning siêu tham số)
- Tuning các tham số chính cho Random Forest (mẫu), có thể lặp cho các mô hình khác

In [6]:
param_grid_rf = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5]
}
gs_rf = GridSearchCV(Pipeline([
    ('preprocess', preprocess),
    ('model', RandomForestClassifier(class_weight='balanced_subsample', random_state=42, n_jobs=-1))
]), param_grid_rf, cv=3, scoring='roc_auc', n_jobs=-1)
gs_rf.fit(X_train, y_train)
print('Best params RF:', gs_rf.best_params_)
print('Best AUC RF:', gs_rf.best_score_)

Best params RF: {'model__max_depth': None, 'model__min_samples_split': 5, 'model__n_estimators': 300}
Best AUC RF: 0.8239739965305762


## 7. Kết luận và gợi ý
- Chọn mô hình tốt nhất dựa trên AUC/Accuracy
- Điều chỉnh threshold nếu cần cân bằng Precision/Recall
- Lưu mô hình và gợi ý can thiệp cho nhóm nguy cơ cao

## 8. Bảng kết quả tổng quan
- Tính Accuracy, Precision, Recall, F1, AUC-ROC cho các mô hình chính
- Có thêm SVM (RBF); XGBoost nếu thư viện `xgboost` sẵn có

In [7]:
from sklearn.svm import SVC

results = []

models = {
    'Logistic Regression': pipe_lr,
    'Decision Tree': pipe_dt,
    'Random Forest': pipe_rf,
    'SVM (RBF)': Pipeline([
        ('preprocess', preprocess),
        ('model', SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42))
    ])
}

# Thêm XGBoost nếu có
try:
    from xgboost import XGBClassifier
    models['XGBoost'] = Pipeline([
        ('preprocess', preprocess),
        ('model', XGBClassifier(
            n_estimators=300,
            max_depth=5,
            learning_rate=0.1,
            subsample=0.9,
            colsample_bytree=0.9,
            objective='binary:logistic',
            eval_metric='auc',
            random_state=42
        ))
    ])
except ImportError:
    pass

for name, pipe in models.items():
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    y_proba = pipe.predict_proba(X_test)[:, 1] if hasattr(pipe, 'predict_proba') else None
    acc = accuracy_score(y_test, y_pred)
    prec = precision_score(y_test, y_pred, zero_division=0)
    rec = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    auc = roc_auc_score(y_test, y_proba) if y_proba is not None else np.nan
    results.append({
        'Model': name,
        'Accuracy': acc,
        'Precision': prec,
        'Recall': rec,
        'F1': f1,
        'AUC-ROC': auc
    })

df_results = pd.DataFrame(results).set_index('Model')
display(df_results.round(3))

# Xuất CSV nếu cần chèn vào báo cáo
df_results.to_csv('model_results_summary.csv')

,Accuracy,Precision,Recall,F1,AUC-ROC
Model,,,,,
Logistic Regression,0.750,0.140,0.80,0.238,0.841
Decision Tree,0.935,0.214,0.12,0.154,0.549
Random Forest,0.950,0.000,0.00,0.000,0.789
SVM (RBF),0.773,0.127,0.62,0.211,0.780
XGBoost,0.950,0.429,0.06,0.105,0.798


## 9. Đánh giá & so sánh (gợi ý viết báo cáo)
- Mô hình tốt nhất theo AUC/Accuracy: ... (điền sau khi chạy)
- So sánh Precision/Recall/F1 giữa Logistic vs RF vs SVM; nêu ưu/nhược điểm (RF mạnh về phi tuyến, Logistic dễ giải thích, SVM tốt khi biên phân tách).
- Nếu XGBoost chạy được: kiểm tra AUC/Recall; đề cập thời gian huấn luyện.
- Đề xuất ngưỡng phân loại khác 0.5 nếu cần tăng Recall cho nhóm nguy cơ cao; cân nhắc sử dụng Precision-Recall trade-off.